In [19]:
from pandas import read_csv
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt


In [20]:
import pandas as pd
from data import citylearn_challenge_2022_phase_1 as competition_data
import os.path as osp

nsload_data_path = osp.join(osp.dirname(competition_data.__file__), "load_data.csv")
nsload_data = pd.read_csv(nsload_data_path)

# for i in range(5):
#     plt.plot(nsload_data["non_shiftable_load"][i*8760 : 8760 + i*8760])
#     plt.show()

In [21]:
selected_params = ["outdoor_dry_bulb", "non_shiftable_load", "non_shiftable_load_future"] # "outdoor_dry_bulb", "outdoor_relative_humidity",
nsload_data = nsload_data[selected_params]

In [22]:
nsload_data

,outdoor_dry_bulb,non_shiftable_load,non_shiftable_load_future
0,20.1,0.851167,0.834600
1,19.7,0.834600,0.838167
2,19.3,0.838167,1.478433
3,18.9,1.478433,1.256200
4,18.3,1.256200,1.869517
...,...,...,...
43785,21.1,3.964600,2.730183
43786,20.6,2.730183,0.935833
43787,21.1,0.935833,2.155017
43788,21.7,2.155017,2.316400


In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# nsload_data = nsload_data[0:8760]
df_load_scaled = sc.fit_transform(nsload_data)
df_load_scaled = np.delete(df_load_scaled, -1, axis=1)
print(df_load_scaled)
print(df_load_scaled.shape)

sc2 = StandardScaler()
df_load_scaled_y = sc2.fit_transform(nsload_data[["non_shiftable_load_future"]])
print(df_load_scaled_y.shape)

[[ 0.91549546 -0.24193832]
 [ 0.80328076 -0.26057443]
 [ 0.69106605 -0.25656223]
 ...
 [ 1.19603224 -0.14669546]
 [ 1.3643543   1.22478309]
 [ 1.25213959  1.40632574]]
(43790, 2)
(43790, 1)


In [24]:
hops = 24
no_records = df_load_scaled_y.shape[0]
no_cols = len(selected_params) - 1
X_train = []
y_train = []
for i in range(hops, no_records):
    X_train.append(df_load_scaled[i-hops:i])
    y_train.append(df_load_scaled_y[i][0])
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train.shape)
print(y_train.shape)

(43766, 24, 2)
(43766,)


In [25]:
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

In [26]:
X_train_reshaped.shape

(43766, 24, 2)

In [27]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model=Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(hops,no_cols)))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train_reshaped, y_train, validation_split=0.5, epochs=300, batch_size=24, shuffle=True, workers=24, use_multiprocessing=True)

Epoch 1/300
912/912 [==============================] - 45s 45ms/step - loss: 0.8121 - val_loss: 0.5526
Epoch 2/300
912/912 [==============================] - 41s 45ms/step - loss: 0.7559 - val_loss: 0.5632
Epoch 3/300
911/912 [============================>.] - ETA: 0s - loss: 0.7385

In [ ]:
from pickle import dump
model.save("non_shiftable_load_model.h5")
dump(sc, open("non_shiftable_load_scaler.pkl", "wb"))

In [60]:
# units = dimensionality of the output space


def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape,
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

demo_model = create_RNN(hidden_units=2, dense_units=1, input_shape=(3,1), activation=['relu', 'relu'])